In [33]:
import tweepy
import json
import time
import datetime
import os

In [20]:
# 180 request / 15 mins window

properties = dict(line.strip().split('=') 
          for line in open('elections.properties') 
          if not line.startswith('#') and not line.startswith('\n'))

print properties

auth = tweepy.OAuthHandler(properties["CONSUMER_KEY"], properties["CONSUMER_SECRET"])
auth.set_access_token(properties["OAUTH_TOKEN"], properties["OAUTH_TOKEN_SECRET"])
api = tweepy.API(auth)

{'CONSUMER_SECRET': '5Ifz6t0UPa6BxaaVm7eTQF4cOe3XcpVsX535nFkaLqy5bp7ehg', 'OAUTH_TOKEN': '594338908-f0JBTAwwVGoZErJ4f1YNAgkj2gws7FQaFur4r2gs', 'CONSUMER_KEY': 'aF03g35xEjOm5OQuFw8PSuwLF', 'OAUTH_TOKEN_SECRET': 'EOR9ptKEtAc0LyhsEOAxsyEshWdgTcJKzXS6iBln2iB6h'}


In [ ]:
QUERIES_DIR = 'queries/'

# Example Query
query = "#DonaldTrump OR #HillaryClinton"

tweets_container = {}


# Code retrieved from
# https://github.com/tweepy/tweepy/issues/197
# I believe the count=100 is unessecary in our case
#
#
# Cursor params are passed to the method (ex.api.search)
# Cursor will alow as to continue to retrieve tweets without worrying about hitting a page limit
#
# api.search arguments are:
#
#     :reference: https://dev.twitter.com/rest/reference/get/search/tweets
#     :allowed_param:'q', 'lang', 'locale', 'since_id', 'geocode',
#      'max_id', 'since', 'until', 'result_type', 'count',
#       'include_entities', 'from', 'to', 'source']
#
# To handle the rate limit:
# # http://docs.tweepy.org/en/v3.5.0/code_snippet.html#handling-the-rate-limit-using-cursors

def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
            
        #except tweepy.RateLimitError as e:
        # I think RateLimitError was depreceated, need to find the new one
        except Exception as e:
            print "Rate Limit Error: %s" % e
            # can handle in any way we want
            time.sleep(15 * 60)


# tweepy_cursor = tweepy.Cursor(api.search,
#                            q=query,
#                            count=100,
#                            result_type="recent",
#                            include_entities=True,
#                            lang="en")            
            
for tweet in limit_handled(tweepy_cursor.items()):
    # converting tweet to json
    tweet_json = json.dumps(tweet._json)
    tweet_json = json.loads(tweet_json)
    tweets_container[tweet_json['id']] = tweet_json
    
    # Test how many tweets we can store
    size = len(tweets_container.keys())
    if size % 500 == 0:
        print size
        
print count

current_time = datetime.datetime.now()

query_dir = QUERIES_DIR + query + '/'
if not os.path.exists(query_dir):
    os.makedirs(query_dir)

with open(query_dir + current_time.isoformat() + '.json', 'w') as f:
    json.dump(countries_place_id, f)

In [8]:
print [(tweet_id, content) for tweet_id, content in tweets_container.iteritems()][:2]

[(794130135637049344, {u'contributors': None, u'truncated': False, u'text': u"RT @janicaeigo: 8 year old American girl's school report on presidential election. \nTrump versus cancer.\n#HillaryClinton https://t.co/uK5uW\u2026", u'is_quote_status': False, u'in_reply_to_status_id': None, u'id': 794130135637049344, u'favorite_count': 0, u'source': u'<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', u'retweeted': False, u'coordinates': None, u'entities': {u'symbols': [], u'user_mentions': [{u'indices': [3, 14], u'screen_name': u'janicaeigo', u'id': 3223393819, u'name': u'Janica Southwick', u'id_str': u'3223393819'}], u'hashtags': [{u'indices': [105, 120], u'text': u'HillaryClinton'}], u'urls': []}, u'in_reply_to_screen_name': None, u'id_str': u'794130135637049344', u'retweet_count': 4, u'in_reply_to_user_id': None, u'favorited': False, u'retweeted_status': {u'contributors': None, u'truncated': False, u'text': u"8 year old American girl's school report on 

In [30]:
# Get place_id for a selection of countries
api = tweepy.API(auth)

countries = ["USA", "Canada", "France", "Denmark", "Mexico", "Brazil", "Germany", "India", "China", "South Africa", 
             "United Kingdom", "Russia", "Australia", "Cuba", "Sweden",
             "Argentina", "Japan", "Spain", "Italy", "Norway", "Finland", "UAE"] # Last line not taken (only first 15)

countries_place_id = []

# i = 0
# while i < len(countries):
#     country = countries[i]

for country in countries[:15]:
    try:
        # TODO Need to lang=en to this query to get the results in English, and not f***ing Danish 
        places = api.geo_search(query=country, granularity="country", )
    except Exception as e:
        print "Rate Limit Error: %s" % e
        time.sleep(15 * 60 + 15)
        continue
        
    for place in places:
        place_id = place.id
        try:
            country_name = place.name.decode('utf-8')
        except Exception as e:
            country_name = place.name
        
        country_place_id = {"real_name": country, "twitter_name": country_name, "place_id": place_id}
        countries_place_id.append(country_place_id)
        
#     i += 1

with open('places_id.json', 'w') as f:
    json.dump(countries_place_id, f)

In [32]:
for country in countries_place_id:
    print country["twitter_name"]

USA
United States Minor Outlying Islands
Canada
Fransk Guyana
Frankrig
French Polynesia
French Southern and Antarctic Lands
Danmark
Mexico
Brasilien
Tyskland
Indien
British Indian Ocean Territory
中华人民共和国
South Africa
South Georgia and the South Sandwich Islands
Central African Republic
Storbritannien
USA
Forenede Arabiske Emirater
United States Minor Outlying Islands
المملكة الأردنية الهاشمية
Saudi Arabien
Hviderusland
Россия
Australia
Cuba
Sverige


In [35]:
current_time = datetime.datetime.now()

print current_time.isoformat()

2016-11-04T18:15:56.788111
